<a href="https://colab.research.google.com/github/datle2403/datle2403/blob/main/titainic_with_Estimator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
df= pd.read_csv('train.csv')

In [3]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [17]:
df['Embarked'].unique()

array(['S', 'C', 'Q', nan], dtype=object)

In [19]:
delete_col=['PassengerId','Name','Ticket']
categorical_col=['Pclass','Sex','SibSp','Parch','Cabin','Embarked']
bucktized_col=['Age']
Numeric_col=['Fare']
missing_col=['Age','Cabin','Embarked']

In [20]:
# refill missing value
for x in missing_col:
  print(df[x].isnull().sum())

177
687
2


In [28]:
# for Embarked
df=df.drop(delete_col,axis=1)
df=df.dropna(subset=['Embarked'])

In [43]:
# for age and Cabin
from sklearn.impute import SimpleImputer
si= SimpleImputer(missing_values=np.nan, strategy='mean')
df['Age']=si.fit_transform(df['Age'].values.reshape(-1,1))
si2= SimpleImputer(missing_values=np.nan, strategy='constant', fill_value='unknown')
df['Cabin']=si2.fit_transform(df['Cabin'].values.reshape(-1,1))

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 889 entries, 0 to 890
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  889 non-null    int64  
 1   Pclass    889 non-null    int64  
 2   Sex       889 non-null    object 
 3   Age       889 non-null    float64
 4   SibSp     889 non-null    int64  
 5   Parch     889 non-null    int64  
 6   Fare      889 non-null    float64
 7   Cabin     889 non-null    object 
 8   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(3)
memory usage: 69.5+ KB


In [60]:
from sklearn.preprocessing import StandardScaler
sc= StandardScaler()
df['Fare']=si.fit_transform(df['Fare'].values.reshape(-1,1))

In [61]:
from sklearn.model_selection import train_test_split
df_train, df_test=train_test_split(df,train_size=0.9)

In [62]:
numeric_feature=[]
numeric_feature.append(tf.feature_column.numeric_column(key='Fare'))

In [85]:
feature_age= tf.feature_column.numeric_column(key='Age')
bucketized_feature=[]
bucketized_feature.append(tf.feature_column.bucketized_column(source_column=feature_age, boundaries=[20,45,70]))
bucketized_feature


[BucketizedColumn(source_column=NumericColumn(key='Age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), boundaries=(20, 45, 70))]

In [86]:
m=[]
for x in categorical_col:
  m.append(list(df[x].unique()))

In [87]:
categorical_indicator_feature=[]
for x,y in zip(categorical_col,m):
  feature=tf.feature_column.categorical_column_with_vocabulary_list(key=x,vocabulary_list=y)
  categorical_indicator_feature.append(tf.feature_column.indicator_column(feature))

In [88]:
def train_input_fn(df_train, batch_size):
  df=df_train.copy()
  X=df.iloc[:,1:]
  y=df.iloc[:,0]
  datasets=tf.data.Dataset.from_tensor_slices((dict(X),y))
  datasets=datasets.shuffle(1000).repeat().batch(batch_size)
  return datasets
def eval_input_fn(df_valid, batch_size):
  df=df_test.copy()
  X=df.iloc[:,1:]
  y=df.iloc[:,0]
  datasets=tf.data.Dataset.from_tensor_slices((dict(X),y))
  datasets=datasets.batch(batch_size)
  return datasets

In [89]:
# step2
all_feature_column= (numeric_feature + bucketized_feature + categorical_indicator_feature)

regressor= tf.estimator.DNNClassifier(feature_columns= all_feature_column, hidden_units=[32,10], model_dir='models/dnn-titanic/')


In [90]:
epochs=200
batch_size=8
step_size= epochs * np.ceil(len(df_train)/ batch_size)

regressor.train(input_fn=lambda: train_input_fn(df_train,8), steps=step_size)

eval_result= regressor.evaluate(input_fn=lambda: eval_input_fn(df_test,8))

In [91]:
eval_result

{'accuracy': 0.8651685,
 'accuracy_baseline': 0.5955056,
 'auc': 0.96016765,
 'auc_precision_recall': 0.94108856,
 'average_loss': 0.3544521,
 'label/mean': 0.40449437,
 'loss': 0.3818811,
 'precision': 0.9,
 'prediction/mean': 0.40262634,
 'recall': 0.75,
 'global_step': 60000}